In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


In [3]:
test_returnDFaboutDifferenceBetweenInput2DFs()

warnings.simplefilter("ignore")

benchmark_name: str = "cg"

classes: list[str] = ["A", "B", "C", "D", "E", "F"]
processes: list[int] = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP: dict[str, str] = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    # "modelLinearSumOf2elementCombination": "説明変数の要素の組み合わせの積の総和",
    # "modelBasicTree": "回帰木",
}

model_names_list_JP: list[str] = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list: list[str] = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date: str = "2022年3月30日"

# コア数と問題サイズの両方を振る

rawDF: pd.DataFrame = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var: list[str] = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var: list[str] = ["functionCallNum"]

print(f"exp_var = {exp_var}")

returned_MAPE_table: pd.DataFrame = return_MAPE_Table_DF_from_rawDF(
    rawDF=rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)
df: pd.DataFrame = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

dfBefore = addLowestMAPEsModelNameColumn(
    df, version=2, model_name_list=model_names_list_JP
)

benchmark_name: str = "cg"

classes: list[str] = ["A", "B", "C", "D", "E", "F"]
processes: list[int] = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP: dict[str, str] = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelLinearSumOf2elementCombination": "説明変数の要素の組み合わせの積の総和",
    # "modelBasicTree": "回帰木",
}

model_names_list_JP: list[str] = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list: list[str] = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date: str = "2022年3月30日"

# コア数と問題サイズの両方を振る

rawDF: pd.DataFrame = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var: list[str] = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var: list[str] = ["functionCallNum"]

print(f"exp_var = {exp_var}")

returned_MAPE_table: pd.DataFrame = return_MAPE_Table_DF_from_rawDF(
    rawDF=rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)
df: pd.DataFrame = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

dfAfter = addLowestMAPEsModelNameColumn(
    df, version=2, model_name_list=model_names_list_JP
)


df = returnDFaboutDifferenceBetweenInput2DFs(dfBefore, dfAfter)
model_names_list: list[str] = df.columns.tolist()

addLowestMAPEsModelNameColumn(df, model_names_list, version=2)

exp_var = ['process', 'na', 'nonzer', 'niter', 'shift']
exp_var = ['process', 'na', 'nonzer', 'niter', 'shift']


,プロセス数を問題サイズで割る,問題サイズをプロセス数で割る,線形,線形＆反比例,説明変数の要素の組み合わせの積の総和,最適モデル
関数名,,,,,,
CONJ_GRAD,37.550135,42.757190,13.161174,12.412559,9.666397,説明変数の要素の組み合わせの積の総和
ICNVRT,159.614199,119.984610,502.123230,6621.011052,84.751964,説明変数の要素の組み合わせの積の総和
MPI_Irecv(),208.955855,56.476064,34.989183,35.515528,22.272503,説明変数の要素の組み合わせの積の総和
MPI_Send(),208.955855,56.476064,34.989183,35.515528,22.272503,説明変数の要素の組み合わせの積の総和
MPI_Wait(),208.955855,56.476064,34.989183,35.515528,22.272503,説明変数の要素の組み合わせの積の総和
SPRNVC,153.544935,182.716358,198.005854,2573.964383,62.480987,説明変数の要素の組み合わせの積の総和
VECSET,153.544935,182.716358,198.005854,2573.964383,62.480987,説明変数の要素の組み合わせの積の総和
